In [37]:
import pandas as pd
import numpy as np
import requests
import os
import re
import nltk
import requests
from bs4 import BeautifulSoup
nltk.download('cmudict')
from nltk.corpus import cmudict, stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#urls=pd.read_csv('Input.xlsx.csv')

In [3]:
def fectchSaveToFile(url,path):
    r=requests.get(url)
    with open(path,'w', encoding="utf-8") as f:
        f.write(r.text)

Getting Stop words


In [32]:
def stop_words_f():
    text_data=''
    files=os.listdir('stop/')
    for file in files:
        with open('stop/'+file,'r',encoding='latin-1') as f:
            text_data+=f.read()
    text_data=re.sub(r'[^\w\s\-]','',text_data).lower()
    stop_words=text_data.split()
    return stop_words

Getting Positive words

In [24]:
def Postive_words(stop_words):
    with open("neg_pos_words/positive-words.txt",'r') as f:
        pos=f.read()
    pos=pos.split()

    pos_words=[]
    for i in pos:
        if i not in stop_words:
            pos_words.append(i)
    return pos_words,pos

Getting negative words

In [34]:
def Negative_words(stop_words):
    with open("neg_pos_words/negative-words.txt",'r',encoding='latin-1') as f:
        neg=f.read()
    neg=neg.split()

    #removing positive words that are atop words
    neg_words=[]
    for i in neg:
        if i not in stop_words:
            neg_words.append(i)
    return neg_words,neg

Getting Web articel

In [7]:
def WebArticle(add):
    with open(add,'r', encoding="utf8") as f:
        html_doc=f.read()

    soup=BeautifulSoup(html_doc,'html.parser')
    # print(soup.title.string)
    web_article=""
    web_article+=soup.title.text
    articles=soup.article
    scrapped=articles.find_all('p')
    for i in scrapped:
        web_article+=i.text
    # web_article=web_article.lower()
    og_web=web_article
    web_article=re.sub(r'[^\w\s\-]','',web_article).lower().split()
    return web_article,og_web

Removing Stop Words

In [8]:
def Remove_stop_words(web_article,stop_words):
    cleaned=[]
    for i in web_article:
        if i not in stop_words:
            cleaned.append(i)
    return cleaned

Calculating Positive and negative words

In [9]:
def positive_negative_score(cleaned,pos,neg):
    p_score=0
    n_score=0
    for i in cleaned:
        if i in pos:
            p_score+=1
        if i in neg:
            n_score+=1
    return p_score,n_score

Calculating Polarity Score


In [10]:
def Polarity_Scroe(p_score,n_score):
    polarity_score=(p_score-n_score)/((p_score+n_score)+0.000001)
    return polarity_score

Calculating Subjectivity Score

In [11]:
def SubjectivityScore(p_score,n_score,cleaned):
    return (p_score+n_score/(len(cleaned)+0.000001))

#### 2.Analysis of Readability,
#### 3.Average Number of Words Per Sentence,
#### 4.Conplex Word Count

> Average Word Length

In [12]:
def AverageSentLength(web_article,og_web):
    # web_article=web_article.lower()
    total_no_of_words=len(web_article)
    no_of_sent=len(nltk.sent_tokenize(og_web))
    Avg_sent_len=total_no_of_words/no_of_sent
    return Avg_sent_len

>Percentage Of Complex words

In [13]:
# Referred from stack overflow
def syllables(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count += 1
    if count == 0:
        count += 1
    return count

d = cmudict.dict()

def nsyl(word):
    try:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    except KeyError:
        #if word not found in cmudict
        return syllables(word)

#complex words
def ComplexWords(web_words):
    l=[]
    s=set()
    for i in range(len(web_words)):
        l.append(nsyl(web_words[i]))
        if l[i]>=2:
            s.add(web_words[i])
    return s

def ComplexWordPercentage(web_words):
    complex_words_percentage=len(ComplexWords(web_words))/len(web_words)*100
    return round(complex_words_percentage,2)

>Fog Index

In [14]:
#Fog Index
def FogIndex(Avg_sent_len,complex_words_percentage):
    fog_index=0.4*(Avg_sent_len+complex_words_percentage)
    return fog_index

# 5. Word Count

In [15]:
#word count using nltk library
def nltk_word_count(web_article_cleaned):
    nltk_stop_words=stopwords.words('english')
    cleaned_words=[]
    for i in web_article_cleaned:
        if i not in nltk_stop_words:
            cleaned_words.append(i)
    return len(cleaned_words)

# 6.Syllable Count Per Word

In [16]:
def SyllableCount(words):
    sc=0
    for word in words:
        if len(word)>2 and (word[-2:]=='es' or word[-2:]=='ed'):
            continue
        for char in word:
            if char in ['a','e','i','o','u']:
                sc+=1
    return sc/len(words)

# 7.Personal Pronouns


In [17]:
def personal_pronouns(web_article):
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronouns = pronounRegex.findall(web_article)
    return len(pronouns)

# 8.Average Word Count

In [18]:
def Avg_word_len(web_article_cleaned):
    sum_of_len=0
    for word in web_article_cleaned:
        sum_of_len+=len(word)
    return sum_of_len/len(web_article_cleaned)

In [39]:
if __name__=='__main__':
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.107 Safari/537.36' }
    urls=pd.read_csv('Input.xlsx.csv')
    n=len(urls)
    # initializing all VARIABLES
    P_Score=np.full(n,fill_value=None)
    N_Score=np.full(n,fill_value=None)
    Polarity_Score=np.full(n,fill_value=None)
    Subj_Score=np.full(n,fill_value=None)
    Avg_Sent_Len=np.full(n,fill_value=None)
    P_complex=np.full(n,fill_value=None)
    Fog_Index=np.full(n,fill_value=None)
    Avg_Num_words_Per_Sent=np.full(n,fill_value=None)
    Complex_Word_Count=np.full(n,fill_value=None)
    Word_Count=np.full(n,fill_value=None)
    Syllable_Count=np.full(n,fill_value=None)
    Pronoun_Count=np.full(n,fill_value=None)
    Avg_Word_Len=np.full(n,fill_value=None)

    stop_words=stop_words_f()
    pos_words,pos=Postive_words(stop_words)
    neg_words,neg=Negative_words(stop_words)
    for i in range(len(urls)):
        id=urls['URL_ID'][i]
        url=urls['URL'][i]
        add="data"+id+".html"
        if requests.get(url, headers=headers).status_code!=404:
            fectchSaveToFile(url,add)
            web_article,og_web=WebArticle(add)
            cleaned=Remove_stop_words(web_article,stop_words)
            #Sentiment Analysis
            p_score,n_score=positive_negative_score(cleaned,pos,neg)
            P_Score[i]=p_score
            N_Score[i]=n_score
            polarity_score=Polarity_Scroe(p_score,n_score)
            Polarity_Score[i]=polarity_score
            subj_score=SubjectivityScore(p_score,n_score,cleaned)
            Subj_Score[i]=subj_score
            #Analysis Of Readability
            avg_sent_len=AverageSentLength(web_article,og_web)
            Avg_Sent_Len[i]=avg_sent_len
            complex_words_percentage=ComplexWordPercentage(web_article)
            P_complex[i]=complex_words_percentage
            fog_index=FogIndex(avg_sent_len,complex_words_percentage)
            Fog_Index[i]=fog_index
            #Average Number of Words Per Sentence
            Avg_Num_words_Per_Sent[i]=avg_sent_len
            #Complex Word Count
            complex_word_count=len(ComplexWords(web_article))
            Complex_Word_Count[i]=complex_word_count
            #Word Count
            word_count=nltk_word_count(web_article)
            Word_Count[i]=word_count
            #Syllables Count PEr Word
            syllablecount=SyllableCount(web_article)
            Syllable_Count[i]=syllablecount
            #Personal Pronouns
            pronouns=personal_pronouns(og_web)
            Pronoun_Count[i]=pronouns
            #Average Word Length
            avg_word_len=Avg_word_len(web_article)
            Avg_Word_Len[i]=avg_word_len

    urls['POSITIVE SCORE']=P_Score
    urls['NEGATIVE SCORE']=N_Score
    urls['POLARITY SCORE']=Polarity_Score
    urls['SUBJECTIVITY SCORE']=Subj_Score
    urls['AVG SENTENCE LENGTH']=Avg_Num_words_Per_Sent
    urls['PERCENTAGE OF COMPLEX WORDS']=P_complex
    urls['FOG INDEX']=Fog_Index
    urls['AVG NUMBER OF WORDS PER SENTENCE']=Avg_Num_words_Per_Sent
    urls['COMPLEX WORD COUNT']=Complex_Word_Count
    urls['WORD COUNT']=Word_Count
    urls['SYLLABLE PER WORD']=Syllable_Count
    urls['PERSONAL PRONOUNS']=Pronoun_Count
    urls['AVG WORD LENGTH']=Avg_Word_Len

    submission=urls[['URL_ID','URL','POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE',\
                     'AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE',\
                       'COMPLEX WORD COUNT', 'WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH']]


In [40]:
submission.to_csv('Output Data Strucute.csv',index=False)